# Identifier les biais en IA
Ce tutoriel vous permet d'entraîner un modèle avec des données réelles et de s'entraîner à identifier les biais.
### Introduction
Civil Comments était une plateforme de commentaires communautaires conçue pour améliorer la qualité et la civilité des discussions en ligne. La plateforme introduisait un modèle de modération collective mêlant interaction humaine et apprentissage automatique. Avant de publier un commentaire, chaque utilisateur devait :
  - Évaluer la courtoisie d’un autre commentaire (par un pouce vers le haut ou le bas) ;
  - Juger s’il s’agissait d’un bon commentaire(« good / sorta / bad »).
  
Civil Comments fut une expérience pionnière de “Civic Tech”. En fermant en 2017, la plateforme a ouvert publiquement sa base de données (1,8 million de commentaires) afin de soutenir la recherche sur la toxicité du langage et les biais en ligne.

Le jeu de données Civil Comments, maintenu depuis par Jigsaw et Google, est aujourd’hui une référence dans les études de détection automatique de propos toxiques et d’analyse de polarisation numérique.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Get the same results each time
np.random.seed(0)

# Load the training data
data = pd.read_csv("data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

# Break into training and test sets
comments_train, comments_test, y_train, y_test = train_test_split(comments, target, test_size=0.30, stratify=target)

# Get vocabulary from training data
vectorizer = CountVectorizer()
vectorizer.fit(comments_train)

# Get word counts for training and test sets
X_train = vectorizer.transform(comments_train)
X_test = vectorizer.transform(comments_test)

# Preview the dataset
data.head()
print("Sample toxic comment:", comments_train.iloc[22])
print("Sample not-toxic comment:", comments_train.iloc[17])

Sample toxic comment: Too dumb to even answer.
Sample not-toxic comment: No they aren't.


Exécutez la cellule de code suivante afin d'utiliser les données pour entraîner un modèle simple. Le résultat montre la précision du modèle sur certaines données de test.

In [2]:
from sklearn.linear_model import LogisticRegression

# Train a model and evaluate performance on test dataset
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

# Function to classify any string
def classify_string(string, investigate=False):
    prediction = classifier.predict(vectorizer.transform([string]))[0]
    if prediction == 0:
        print("NOT TOXIC:", string)
    else:
        print("TOXIC:", string)

Accuracy: 0.9304755967877966


Environ 93 % des commentaires dans les données de test sont classés correctement.

### 1) Essayer le modèle
Utiliser la cellule suivante pour écrire vos propres commentaires et les transmettre au modèle, et vérifier (ou pas) que le modèle les classe comme toxiques.

In [3]:
# Comment to pass through the model
my_comments = [
    "I love apples", 
    "This guy is a perfect asshole", 
    "I been there, it's awesome", 
    "The referee is shit!", 
    "You are a fucking cunt, lady"
]
results = [classify_string(c) for c in my_comments]

NOT TOXIC: I love apples
TOXIC: This guy is a perfect asshole
NOT TOXIC: I been there, it's awesome
TOXIC: The referee is shit!
TOXIC: You are a fucking cunt, lady


Une fois les commentaires testés, nous allons comprendre comment le modèle prend des décisions dans la cellule de code suivante.

Le modèle attribue un coefficient à chacun des quelque 58 000 mots, les coefficients les plus élevés indiquant les mots jugés les plus toxiques par le modèle. La cellule de code affiche les dix mots considérés comme les plus toxiques, ainsi que leurs coefficients.

In [4]:
coefficients = pd.DataFrame({"word": sorted(list(vectorizer.vocabulary_.keys())), "coeff": classifier.coef_[0]})
coefficients.sort_values(by=['coeff']).tail(10)

,word,coeff
20745,fools,6.317838
34211,moron,6.357154
16844,dumb,6.399255
12907,crap,6.533120
38317,pathetic,6.603762
25850,idiotic,7.061440
49802,stupidity,7.617183
25858,idiots,8.694790
25847,idiot,8.701940
49789,stupid,9.318078


### 2) Mots les plus toxiques
Regardez les mots les plus toxiques de la cellule de code ci-dessus. Êtes-vous surpris d'en voir certains ? Y a-t-il des mots qui ne devraient pas figurer dans la liste ?

### 3) Analyse plus approfondie
Examinons de plus près comment le modèle classe les commentaires.
   1. Commencez par exécuter la cellule de code telle quelle pour classer le commentaire « I have a christian friend ». Vous devriez constater qu'il a été classé comme « NON TOXIQUE ». De plus, vous pouvez voir les scores attribués à certains mots. Notez que tous les mots du commentaire n'apparaîtront probablement pas.
   2. Ensuite, essayez « I have a muslim friend », puis « I have a white friend » et « I have a black friend ».
N'hésitez pas à tester d'autres commentaires pour voir comment le modèle les classe.

In [21]:
# Set the value of new_comment
new_comments = [
    "I have a christian friend", 
    "I have a muslim friend", 
    "I have a white friend", 
    "I have a black friend",
    "Today, I saw a black friend",
    "Pigs are great animals",
    "Cows are great animals",
    "Rats are tiny animals and they are often misunderstood.",
    "You are a rat",
    "Fuck"
]

results = [(c, classify_string(c), coefficients[coefficients.word.isin(c.split())])for c in new_comments]

NOT TOXIC: I have a christian friend
TOXIC: I have a muslim friend
NOT TOXIC: I have a white friend
TOXIC: I have a black friend
NOT TOXIC: Today, I saw a black friend
TOXIC: Pigs are great animals
NOT TOXIC: Cows are great animals
TOXIC: Rats are tiny animals and they are often misunderstood.
TOXIC: You are a rat
TOXIC: Fuck


### 4) Identifier les biais
Voyez-vous des signes de biais potentiel dans le modèle ? Dans la cellule de code ci-dessus,
- Comment le modèle a-t-il classé « I have a christan friend » et « I have a muslim friend » ?
- Comment a-t-il classé « I have a white friend » et « I have a black friend » ?
Une fois la réponse obtenue, exécutez la cellule de code suivante.

### 5) Testez votre compréhension 
Nous allons nous éloigner des données du concours Jigsaw et envisager un scénario similaire (mais hypothétique !) où vous travaillez avec un ensemble de données de commentaires en ligne pour entraîner un modèle capable de classer les commentaires comme toxiques.

Vous remarquez que les commentaires faisant référence à l'islam sont plus susceptibles d'être toxiques que ceux faisant référence à d'autres religions, car la communauté en ligne est islamophobe. Quel type de biais cela peut-il introduire dans votre modèle ?

### 6) Testez votre compréhension (suite)
Nous reprenons le même scénario hypothétique : vous essayez d'entraîner un modèle pour classer les commentaires en ligne comme toxiques.

Vous prenez tous les commentaires qui ne sont pas déjà en anglais et vous les traduisez en anglais avec un outil distinct. Ensuite, vous traitez tous les messages comme s'ils avaient été initialement rédigés en anglais. De quel type de biais votre modèle souffrira-t-il ?

### 7) Testez votre compréhension (suite et fin)
Nous reprenons le même scénario hypothétique : vous essayez d'entraîner un modèle pour classer les commentaires en ligne comme toxiques.

L'ensemble de données utilisé pour entraîner le modèle contient principalement des commentaires d'utilisateurs basés au Royaume-Uni.

Après avoir entraîné un modèle, vous évaluez ses performances avec un autre ensemble de données de commentaires, également principalement d'utilisateurs basés au Royaume-Uni, et les performances sont excellentes ! Vous le déployez pour une entreprise basée en Australie, et ses performances sont médiocres en raison des différences entre l'anglais britannique et l'anglais australien. De quels types de biais le modèle souffre-t-il ?